In [1]:
from transformers import AutoTokenizer
import transformers

print('transformers version: ', transformers.__version__)

transformers version:  4.56.1


1. 编解码示例

In [2]:
tokenizer_path = "../mini_tokenizer"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

text = "测试 Mini-Tokenizer 效果。\n特殊字符:\n<|endoftext|> <|im_start|> <|im_end|> <think> </think>"

print("-------- 编码 --------")
input_ids = tokenizer.encode(text, return_tensors="pt")
print("token ids:\n", input_ids)
print("token ids length: ", len(input_ids[0]))

# special token会被跳过, <think></think>作为added token，具有确定的id，但是没有被设置为special token，因此会被保留
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("-------- 解码 --------")
print(decoded_text)

# 直接使用tokenizer，返回input_ids、token_type_ids和attention_mask字段
print("-------- 直接使用tokenizer编码 --------")
encoded_text = tokenizer(text, padding="max_length", max_length=30)  # 会自动补4个pad token
print(encoded_text)

-------- 编码 --------
token ids:
 tensor([[ 2517, 17459,    73,    13,    52,  3168,   346,  1557,   356,   221,
          2301,   286,   199,  2629,  9951,    26,   199,     0,   221, 32000,
           221, 32001,   221, 32002,   221, 32003]])
token ids length:  26
-------- 解码 --------
测试 Mini-Tokenizer 效果。
特殊字符:
   <think> </think>
-------- 直接使用tokenizer编码 --------
{'input_ids': [2517, 17459, 73, 13, 52, 3168, 346, 1557, 356, 221, 2301, 286, 199, 2629, 9951, 26, 199, 0, 221, 32000, 221, 32001, 221, 32002, 221, 32003, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]}


In [3]:
# 分词示例
tokens = tokenizer.tokenize(text)  # 因为是字节级BPE，所以显示结果为乱码
print("分词为tokens:\n", tokens)

# 将tokens转回id
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("token ids:\n", token_ids)

# 将id转回tokens
reconstructed_tokens = tokenizer.convert_ids_to_tokens(token_ids)
print("还原后的tokens:\n", reconstructed_tokens)

# 将tokens转回文本
reconstructed_text = tokenizer.decode(token_ids, skip_special_tokens=False)  #不跳过特殊token
print("还原后的文本:\n", reconstructed_text)

分词为tokens:
 ['æµĭè¯ķ', 'ĠMin', 'i', '-', 'T', 'ok', 'en', 'iz', 'er', 'Ġ', 'æķĪæŀľ', 'ãĢĤ', 'Ċ', 'çī¹æ®Ĭ', 'åŃĹç¬¦', ':', 'Ċ', '<|endoftext|>', 'Ġ', '<|im_start|>', 'Ġ', '<|im_end|>', 'Ġ', '<think>', 'Ġ', '</think>']
token ids:
 [2517, 17459, 73, 13, 52, 3168, 346, 1557, 356, 221, 2301, 286, 199, 2629, 9951, 26, 199, 0, 221, 32000, 221, 32001, 221, 32002, 221, 32003]
还原后的tokens:
 ['æµĭè¯ķ', 'ĠMin', 'i', '-', 'T', 'ok', 'en', 'iz', 'er', 'Ġ', 'æķĪæŀľ', 'ãĢĤ', 'Ċ', 'çī¹æ®Ĭ', 'åŃĹç¬¦', ':', 'Ċ', '<|endoftext|>', 'Ġ', '<|im_start|>', 'Ġ', '<|im_end|>', 'Ġ', '<think>', 'Ġ', '</think>']
还原后的文本:
 测试 Mini-Tokenizer 效果。
特殊字符:
<|endoftext|> <|im_start|> <|im_end|> <think> </think>


2. 特殊token示例

In [4]:
print("special token:\n", tokenizer.special_tokens_map)
print("special token ids:\n", tokenizer.all_special_ids)
print("unknow token id:", tokenizer.unk_token_id)
print("think token id:\n<think>:", tokenizer.encode("<think>"), "\n</think>:", tokenizer.encode("</think>"))
print(f"vocab size: {len(tokenizer)}")

special token:
 {'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}
special token ids:
 [32001, 0, 32000]
unknow token id: None
think token id:
<think>: [32002] 
</think>: [32003]
vocab size: 32004


3. 对话模板示例

In [5]:

# 测试对话-会自动格式化think结构
print("======================== 对话模板测试 ========================")
print("1. 参数设置: tokenize=False, add_generation_prompt=False")
print('-------------------------------------------------------------')
messages = [
    {"role": "system", "content": "你是一个专业的助手"},
    {"role": "user", "content": "你好"},
    {"role": "assistant", "content": "你好，我是一个专业的助手，我会尽力回答你的问题。"},  # 不带 think 标签
    {"role": "user", "content": "你好，请帮我分析这个问题"},
    {"role": "assistant", "content": "<think>这是一个需要分析的问题，让我先思考一下...</think>根据我的分析，答案是..."}  # 带 think 标签
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
print(prompt)
print('-------------------------------------------------------------')

print("2. 参数设置: tokenize=False, add_generation_prompt=True")
print('-------------------------------------------------------------')
messages = [
    {"role": "user", "content": "你好，请帮我分析这个问题"},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  # 对话补全时使用
print(prompt)
print('-------------------------------------------------------------')

print("3. 参数设置: tokenize=True, add_generation_prompt=True")
print('-------------------------------------------------------------')
messages = [
    {"role": "user", "content": "你好，请帮我分析这个问题"},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True)  # 对话补全时使用
print(prompt)

======================== 对话模板测试 ========================
1. 参数设置: tokenize=False, add_generation_prompt=False
-------------------------------------------------------------
<|im_start|>system
你是一个专业的助手<|im_end|>
<|im_start|>user
你好<|im_end|>
<|im_start|>assistant
你好，我是一个专业的助手，我会尽力回答你的问题。<|im_end|>
<|im_start|>user
你好，请帮我分析这个问题<|im_end|>
<|im_start|>assistant
<think>
这是一个需要分析的问题，让我先思考一下...
</think>

根据我的分析，答案是...<|im_end|>

-------------------------------------------------------------
2. 参数设置: tokenize=False, add_generation_prompt=True
-------------------------------------------------------------
<|im_start|>user
你好，请帮我分析这个问题<|im_end|>
<|im_start|>assistant

-------------------------------------------------------------
3. 参数设置: tokenize=True, add_generation_prompt=True
-------------------------------------------------------------
[32000, 616, 356, 199, 2657, 807, 270, 1422, 2280, 537, 1010, 9502, 32001, 199, 32000, 2570, 16415, 199]
